# Import librairies

In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# API request

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# API requests
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 47.21955246182277,
	"longitude": -1.5404244660962094,
	"hourly": ["temperature_2m", "visibility", "precipitation", "wind_speed_10m"],
	"timezone": "Europe/Berlin",
	"start_date": "2025-02-16",
	"end_date": "2025-04-12"
}
responses = openmeteo.weather_api(url, params=params)

# Process location
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_visibility = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["visibility"] = hourly_visibility
hourly_data["precipitation"] = hourly_precipitation
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)


Coordinates 47.220001220703125°N -1.5399999618530273°E
Elevation 13.0 m asl
Timezone b'Europe/Berlin'b'GMT+2'
Timezone difference to GMT+0 7200 s


In [6]:
hourly_dataframe.head()

,date,temperature_2m,visibility,precipitation,wind_speed_10m
0,2025-02-15 22:00:00+00:00,3.9805,24140.0,0.0,10.144082
1,2025-02-15 23:00:00+00:00,3.2305,24140.0,0.0,11.019764
2,2025-02-16 00:00:00+00:00,2.8305,24140.0,0.0,11.885453
3,2025-02-16 01:00:00+00:00,2.2805,24140.0,0.0,11.019764
4,2025-02-16 02:00:00+00:00,1.8805,24140.0,0.0,10.495713


In [ ]:
#hourly_dataframe.to_csv('AI_model_urban_mobility/data/preprocessed/weather_data.csv', index = False)